# Install the requirments for ENV setup

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.1 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq llava-torch==1.1.1 --progress-bar off
# !pip install gdown

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.2 requires huggingface-hub>=0.21.2, but you have huggingface-hub 0.17.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.1 requires httpx>=0.25.0, but you have httpx 0.24.0 which is incompatible.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires 

# load the lib and modules required for test run of LLMs

In [2]:
import textwrap
from io import BytesIO

import requests
import torch
from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX
from llava.conversation import SeparatorStyle, conv_templates
from llava.mm_utils import (
    KeywordsStoppingCriteria,
    get_model_name_from_path,
    process_images,
    tokenizer_image_token,
)
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from PIL import Image
import matplotlib.pyplot as plt

[2025-06-05 01:06:39,530] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-06-05 01:06:42.778540: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 01:06:42.778694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 01:06:42.909093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
disable_torch_init()
MODEL = "microsoft/llava-med-v1.5-mistral-7b"
model_name = get_model_name_from_path(MODEL)
model_name

'llava-med-v1.5-mistral-7b'

# Load the test images for testing the model

In [4]:
tokenizer, model, image_processor, context_len = load_pretrained_model(
                                                    model_path=MODEL, 
                                                    model_base=None, 
                                                    model_name=model_name, 
                                                    load_8bit=True )

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


You are using a model of type llava_mistral to instantiate a model of type llava. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlavaLlamaForCausalLM were not initialized from the model checkpoint at microsoft/llava-med-v1.5-mistral-7b and are newly initialized: ['model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.25.self

In [5]:
def load_image(image_file):
    if image_file.startswith("http://") or image_file.startswith("https://"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image

In [6]:
def process_image(image):
    args = {"image_aspect_ratio": "pad"}
    image_tensor = process_images([image], image_processor, args)
    return image_tensor.to(model.device, dtype=torch.float16)

In [7]:
def create_prompt(prompt: str):
    CONV_MODE = 'llava_v0'
    conv = conv_templates[CONV_MODE].copy()
    roles = conv.roles
    prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt
    conv.append_message(roles[0], prompt)
    conv.append_message(roles[1], None)
    return conv.get_prompt(), conv

In [8]:
def ask_image(image: Image, prompt: str):
    image_tensor = process_image(image)
    prompt, conv = create_prompt(prompt)
    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .to(model.device)
    )

    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    stopping_criteria = KeywordsStoppingCriteria(
        keywords=[stop_str], tokenizer=tokenizer, input_ids=input_ids
    )

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=0.01,
            max_new_tokens=512,
            use_cache=True,
            stopping_criteria=[stopping_criteria],
        )
    return tokenizer.decode(
        output_ids[0, input_ids.shape[1] :], skip_special_tokens=True
    ).strip()

In [9]:
# Load the saved dataset
data = torch.load('/kaggle/input/peft-thyroid-dataset/augmented_dataset.pt', weights_only=False)

# Print sizes
print("Train images:", data['train_images'].shape)
print("Train labels:", len(data['train_labels']))
print("Train IDs:", len(data['train_ids']))  # list of strings

print("Val images:", data['val_images'].shape)
print("Val labels:", len(data['val_labels']))
print("Val IDs:", len(data['val_ids']))

print("Test images:", data['test_images'].shape)
print("Test labels:", len(data['test_labels']))
print("Test IDs:", len(data['test_ids']))

Train images: torch.Size([1035, 3, 360, 560])
Train labels: 1035
Train IDs: 1035
Val images: (45, 3, 360, 560)
Val labels: 45
Val IDs: 45
Test images: (45, 3, 360, 560)
Test labels: 45
Test IDs: 45


In [10]:
from torch.utils.data import Dataset, DataLoader

def map_label_to_tirads(label):
    if "2" in label: return "TI-RADS 2"
    elif "3" in label: return "TI-RADS 3"
    elif "4" in label: return "TI-RADS 4"
    elif "5" in label: return "TI-RADS 5"
    else: return "Unknown"

class ThyroidDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images

        label_map = {"TI-RADS 2": 0, "TI-RADS 3": 1, "TI-RADS 4": 2, "TI-RADS 5": 3}
        self.labels = []

        for label in labels:
            if not isinstance(label, str):
                raise ValueError(f"Unexpected non-string label: {label}")
            coarse_label = map_label_to_tirads(label)
            if coarse_label not in label_map:
                raise ValueError(f"🚨 Unknown label: {label} → {coarse_label}")
            self.labels.append(label_map[coarse_label])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'pixel_values': self.images[idx],  # tensor
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


# Create Datasets
train_dataset = ThyroidDataset(data['train_images'], data['train_labels'])
val_dataset = ThyroidDataset(data['val_images'], data['val_labels'])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)

In [11]:
from torch import nn
device = "cuda"

# Only add the classifier if it doesn't exist
if not hasattr(model, "classifier"):
    model.classifier = nn.Linear(model.config.hidden_size, 4).to(device)
    nn.init.xavier_uniform_(model.classifier.weight)
    nn.init.zeros_(model.classifier.bias)

# Make sure classifier parameters are trainable
for param in model.classifier.parameters():
    param.requires_grad = True

In [12]:
# Set BitFit — biases and classifier only
for name, param in model.named_parameters():
    if ".bias" in name or "classifier" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [13]:
for name, param in model.named_parameters():
    if param.dtype == torch.float16:
        print(f"Converting {name} from float16 to float32")
        param.data = param.data.float()

Converting model.embed_tokens.weight from float16 to float32
Converting model.layers.0.input_layernorm.weight from float16 to float32
Converting model.layers.0.post_attention_layernorm.weight from float16 to float32
Converting model.layers.1.input_layernorm.weight from float16 to float32
Converting model.layers.1.post_attention_layernorm.weight from float16 to float32
Converting model.layers.2.input_layernorm.weight from float16 to float32
Converting model.layers.2.post_attention_layernorm.weight from float16 to float32
Converting model.layers.3.input_layernorm.weight from float16 to float32
Converting model.layers.3.post_attention_layernorm.weight from float16 to float32
Converting model.layers.4.input_layernorm.weight from float16 to float32
Converting model.layers.4.post_attention_layernorm.weight from float16 to float32
Converting model.layers.5.input_layernorm.weight from float16 to float32
Converting model.layers.5.post_attention_layernorm.weight from float16 to float32
Convertin

In [14]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.numel()}")

In [17]:
from tqdm import tqdm
from torch.nn.functional import cross_entropy

class_labels = ["TI-RADS 2", "TI-RADS 3", "TI-RADS 4", "TI-RADS 5"]
label_to_index = {label: i for i, label in enumerate(class_labels)}

# model.to(device)
model.train()
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5,
    weight_decay=0.0
)
num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"🔍 Trainable parameters: {num_trainable}")

for epoch in range(20):
    total_loss = 0
    for batch in tqdm(val_loader):
        optimizer.zero_grad()

        # Process image
        processed = image_processor(images=batch['pixel_values'], return_tensors="pt").to(device)
        # pixel_values = processed["pixel_values"]
        pixel_values = processed["pixel_values"]

        batch_size = processed["pixel_values"].size(0)
        prompts = ["The TI-RADS classification of the thyroid ultrasound image is:"] * batch_size
        inputs = tokenizer(prompts, return_tensors="pt", padding='longest', truncation=True).to(device)
        
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            images=pixel_values,
            output_hidden_states=True,
            return_dict=True
        )

        # print(batch['labels'])
        true_labels = batch['labels'].to(device)  # Ensure labels are 0–3
        # print("Labels:", true_labels)
        # print("Min label:", true_labels.min().item(), "Max label:", true_labels.max().item())
        assert torch.all((true_labels >= 0) & (true_labels < 4)), "🚨 Invalid label detected!"

        # with torch.no_grad():
        #     hidden_check = outputs.hidden_states[-1][:, -1, :].cpu().numpy()
        #     print("Embed mean/std:", hidden_check.mean(), hidden_check.std())

        hidden = outputs.hidden_states[-1][:, -1, :].float()
        logits = model.classifier(hidden)  # shape: [B, 4]
        loss = cross_entropy(logits, true_labels)
        # print("Logits stats — mean:", logits.mean().item(), "max:", logits.max().item())
        # print("Loss:", loss.item())
        if not torch.isfinite(loss):
            print(f"🚨 Non-finite loss at step {epoch}, skipping.")
            continue
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

🔍 Trainable parameters: 296964


100%|██████████| 45/45 [00:39<00:00,  1.14it/s]


Epoch 1: Loss = 146.3092


100%|██████████| 45/45 [00:39<00:00,  1.14it/s]


Epoch 2: Loss = 116.2920


...

Epoch 18: Loss = 113.8889


100%|██████████| 45/45 [00:39<00:00,  1.13it/s]


Epoch 19: Loss = 113.8870


100%|██████████| 45/45 [00:39<00:00,  1.14it/s]

Epoch 20: Loss = 113.8854


In [ ]:
dtypes = set(p.dtype for p in model.parameters())
print("Model parameter dtypes:", dtypes)

